In [2]:
import json
import pandas as pd
from sentence_transformers import (SentenceTransformer, util)

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
with open('Data/IW_2.0_flows.json','r') as f:
    iw_flows = json.load(f)
iw_embeddings = model.encode(iw_flows)

In [43]:
products = ['NH3']
products_embeddings = model.encode(products)

In [44]:
scores = util.pytorch_cos_sim(products_embeddings, iw_embeddings)
sorted_scores, indices = scores.sort(dim=1, descending=True)

In [45]:
df_results = pd.DataFrame(None, ['order', 'iw_flow', 'similarity'])

number_of_matches = 5

for i, product in enumerate(products):
    for j in range(0, number_of_matches):
        df_results = pd.concat([df_results, 
                   pd.DataFrame([product, 
                                 j+1,
                                 iw_flows[indices[i][j].cpu().numpy()], 
                                 sorted_scores[i][j].cpu().numpy().tolist()],
                                ['flow', 'order', 'iw_flow', 'similarity'])],
                               axis=1)
        
df_results = df_results.T.set_index(['flow', 'order'])

In [46]:
df_results

iw_flow similarity
flow order                        
NH3  1      Ammonia, NI   0.673041
     2      Ammonia, NR   0.666009
     3      Ammonia, NA   0.665262
     4      Ammonia, NE   0.663512
     5          Ammonia   0.651905